In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
#from plotly.offline import init_notebook_mode, iplot
#import plotly.graph_objs as go
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
##### 파일 불러오기 #####
df = pd.read_csv("./data_preprocessing/4_card_spending_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'card_cnt', 'card_amt']  ##### 각자 데이터에 맞게 열이름 변경하기 #####
df

,base_date,emd_cd,card_cnt,card_amt
base_date,,,,
2018-01-31,2018-01-31,50110250,215.794788,8171132.984
2018-02-28,2018-02-28,50110250,213.713262,8605532.333
2018-03-31,2018-03-31,50110250,228.401338,8517373.716
2018-04-30,2018-04-30,50110250,239.895973,8640048.185
2018-05-31,2018-05-31,50110250,250.348387,9384709.810
...,...,...,...,...
2021-02-28,2021-02-28,50130620,65.216000,2463788.712
2021-03-31,2021-03-31,50130620,67.700000,2572585.700
2021-04-30,2021-04-30,50130620,68.642336,2473174.901


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date', 'card_cnt'], 1)     ##### 각자 필요없는 열 제거 #####
data

,emd_cd,card_amt
base_date,,
2018-01-31,50110250,8171132.984
2018-02-28,50110250,8605532.333
2018-03-31,50110250,8517373.716
2018-04-30,50110250,8640048.185
2018-05-31,50110250,9384709.810
...,...,...
2021-02-28,50130620,2463788.712
2021-03-31,50130620,2572585.700
2021-04-30,50130620,2473174.901


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50130620

# base_date, card_cnt 만 갖는 df 생성

sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df.head()

,card_amt
base_date,
2018-01-31,2231735.720
2018-02-28,2471802.058
2018-03-31,2033216.258
2018-04-30,2184983.868
2018-05-31,2364584.957


In [5]:
# series 형으로 변환
sub_df.to_dict('series')
sub_df.head()

,card_amt
base_date,
2018-01-31,2231735.720
2018-02-28,2471802.058
2018-03-31,2033216.258
2018-04-30,2184983.868
2018-05-31,2364584.957


# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)
#sub_df_float.tail()

In [8]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))
    
##### 여기서 p-value 확인 -> 0.05보다 크면 아래로 내려가서 차분하기 / 작으면 차분 생략(아래 셀 생략) #####

ADF Statistic : -3.512807
p-value : 0.007664
Critical Values :
	1%: -3.600983
	5%: -2.935135
	10%: -2.605963


In [9]:
# 차분
diff_1 = sub_df_float.diff(periods=1).iloc[1:]

# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
result = adfuller(diff_1)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))
    
##### 여기서 또 p-value 확인 -> 0.05보다 크면 이 파일 안되고 log파일로 넘어가세요 / 작으면 아래로 쭉 진행 #####

ADF Statistic : -8.547878
p-value : 0.000000
Critical Values :
	1%: -3.605565
	5%: -2.937069
	10%: -2.606986


In [11]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

##### 여기서 아래쪽에 'Best model:  ARIMA(0,1,0)(0,0,0)[1] intercept' 이렇게 나오면 (0,1,0) 가져다 쓸거임 #####
##### ARIMA(0,1,0) 여기 숫자 세 개만 잘 보면 됨 !!!! #####

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=1148.037, Time=0.04 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=1141.501, Time=0.30 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=1135.050, Time=0.03 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=1136.682, Time=0.04 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=1137.325, Time=0.05 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=1138.555, Time=0.14 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=1134.510, Time=0.02 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=1137.906, Time=0.03 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=1135.743, Time=0.07 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=1137.396, Time=0.11 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=1138.694, Time=0.11 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=1134.882, Time=0.03 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=1137.597, Time=0.08 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=1137.546, Time=0.15 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=1139.533, Time=0.32 sec
 ARIMA(3,0

ARIMA(order=(1, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [14]:
####################################
######### order 바꾸기 ############
###################################
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.card_amt.values, order=(1,0,0))    ##### 여기 order=(0,1,0)에 위에서 나온 숫자 넣는거 #####
                                    ##### 그리고 sub_df.card_amt.values에서 가운데 card_amt 각자걸로 바꾸기 #####

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(1, 0)   Log Likelihood                -564.238
Method:                       css-mle   S.D. of innovations         164487.916
Date:                Sat, 11 Sep 2021   AIC                           1134.475
Time:                        18:00:31   BIC                           1139.688
Sample:                             0   HQIC                          1136.386
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        2.56e+06   5.75e+04     44.531      0.000    2.45e+06    2.67e+06
ar.L1.y        0.5711      0.130      4.402      0.000       0.317       0.825
                                    Roots           

C:\Users\pione\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


# 예측하기

In [15]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

##### 맨 앞에 나온 'array([10681271.81014634, 10741037.02029268])' 이게 차례대로 7, 8월 예측값임 #####
##### 7월 : 10681271.81014634, 8월 : 10741037.02029268 #####
##### 이 값 가져다 엑셀파일에 입력하기 #####

(array([2471161.27417736, 2509156.41197109]), array([164487.91630228, 189423.17534839]), array([[2148770.88233285, 2793551.66602186],
       [2137893.81045103, 2880419.01349115]]))
